# Load Configurations & Libraries
Creating Vector Database

In [ ]:
# If we run this machine on GPU machine it will be faster.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd

In [108]:
apipath = r'/content/drive/MyDrive/config/hbqa-colab.txt'
apipath = r'H:\\My Drive\\config\\hbqa.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)

datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']
corpus_sectionpath = config['global']['CORPUS_FOLDER_SECTIONS']
OPENAI_KEY = config['global']['OPENAI_KEY']
PINECONE_KEY = config['global']['PINECONE_KEY']
PINECONE_ENV = config['global']['PINECONE_ENV']
CHATPDF_KEY = config['global']['CHATPDF_KEY']


# PINECONE is Vector Database. To store the vector so that we can quickly search the vector space.
# https://app.pinecone.io
# get PINECONE_API_KEY key from app.pinecone.io
# find your PINECONE_ENVIRONMENT next to the api key in pinecone console

### Load Embedding Model. It will create 384 dim vector for the sentences.

In [ ]:
# !pip install -Uq sentence-transformers

In [ ]:
# https://www.sbert.net/docs/pretrained_models.html

#420MB, all-mpnet-base-v2, Base Model:	https://huggingface.co/microsoft/mpnet-base, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Base Model:	https://huggingface.co/nreimers/MiniLM-L6-H384-uncased Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
#250MB, multi-qa-distilbert-cos-v1',  https://huggingface.co/distilbert-base, Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
##1.36GB, all-roberta-large-v1, https://huggingface.co/microsoft/roberta-large,  Max Sequence Length:	256, Dimensions: 1024, 

modelname = 'sentence-transformers/all-MiniLM-L6-v2'
modelname = 'sentence-transformers/roberta-large'
modelname = 'sentence-transformers/multi-qa-distilbert-cos-v1'



In [4]:
from sentence_transformers import SentenceTransformer, util

#util.dot_score, util.cos_sim
 
embmodel = SentenceTransformer(modelname)

c:\Users\hari_\anaconda3\envs\hbqa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Creating Embedding for Chunks

In [11]:
FileToProcess = '04.2-Chunk_with_Prompts+ChatGPT_Response.csv' #This is based on Chapter 3 & 4.
df_hbqa_chunk= pd.read_csv(datapath+FileToProcess) #This CSV is created from google drive file. It has Chunk info.

print(df_hbqa_chunk.shape)
df_hbqa_chunk = df_hbqa_chunk.loc[df_hbqa_chunk['ChatGPT_Response'].notna(),:]
print(df_hbqa_chunk.shape)

(1697, 9)
(537, 9)


In [9]:
df_hbqa_chunk.head(3)

,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,1106
222,Book03_002,389,4793,808,1075,Even this is eternal morality. They that perfo...,"Write 5 unique questions, corresponding answer...",Question 1: What is the significance of perfor...,5.0
223,Book03_003,390,5453,851,1132,"Section III\n""Vaisampayana said, 'Yudhishthira...","Write 6 unique questions, corresponding answer...",Question 1: What predicament does Yudhishthira...,6.0
224,Book03_004,392,4383,775,1031,"Section IV\n""Vaisampayana said,--""After the Pa...","Write 5 unique questions, corresponding answer...",Question 1: What was the primary concern expre...,5.0
225,Book03_005,393,4183,717,954,"Section V\n""Vaisampayana said, 'Desirous of li...","Write 5 unique questions, corresponding answer...",Question 1: Where did the Pandavas go after le...,5.0
226,Book03_006,394,3765,625,831,"Section VI\n""Vaisampayana said, 'O king, after...","Write 4 unique questions, corresponding answer...",Question 1: Why did Dhritarashtra feel remorse...,4.0
...,...,...,...,...,...,...,...,...,...
1692,Book14_052,2722,1666,271,360,When king Yudhishthira the just of Kuru's race...,"Write 1 unique questions, corresponding answer...",1,NaN
1693,Book14_092,2767,1948,339,451,would do when seeing some injury done to him. ...,"Write 1 unique questions, corresponding answer...",1,NaN
1694,Book15_012,2781,2146,369,491,"Section XII\n""Arjuna said, 'O Bhima, thou art ...","Write 1 unique questions, corresponding answer...",1,NaN
1695,Book15_021,2790,2319,388,516,"Section XXI\n"" Vaisampayana said, 'Upon the re...","Write 1 unique questions, corresponding answer...",1,NaN


In [29]:
df_hbqa_chunk.tail(3)

,index,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,ChunkVector
534,1694,Book15_012,2781,2146,369,491,"Section XII\n""Arjuna said, 'O Bhima, thou art ...","Write 1 unique questions, corresponding answer...",1,"[-0.06720974296331406, 0.091431625187397, -0.0..."
535,1695,Book15_021,2790,2319,388,516,"Section XXI\n"" Vaisampayana said, 'Upon the re...","Write 1 unique questions, corresponding answer...",1,"[0.0037896237336099148, 0.06728169322013855, -..."
536,1696,Book18_002,2825,877,147,196,"""Overwhelmed by sorrow and grief, and with his...","Write 1 unique questions, corresponding answer...",1,"[0.02505647949874401, 0.061731502413749695, 0...."


In [ ]:
df_hbqa_chunk.drop(columns=['1106'], axis=1,inplace=True)

In [14]:
DEVICE='cuda'

In [12]:
%time
Chunk_Sentences = df_hbqa_chunk.Chunk.tolist()
Chunk_Embeddings = embmodel.encode(Chunk_Sentences)

CPU times: total: 0 ns
Wall time: 0 ns


In [14]:
import torch

In [16]:
ChunkVectors= torch.tensor(Chunk_Embeddings, dtype=torch.float)#.to(DEVICE)

In [17]:
# Flatten the tensors into 1D arrays
ChunkVec_list = ChunkVectors.tolist()

In [18]:
df_hbqa_chunk['ChunkVector'] = ChunkVectors.tolist()

In [19]:
df_hbqa_chunk.reset_index(inplace=True)

In [26]:
df_hbqa_chunk.to_csv(datapath + '07.1-HBQA_Chunk_Vector.csv')

In [27]:
print(df_hbqa_chunk.shape)
df_hbqa_chunk.head(3)

(537, 10)


,index,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,ChunkVector
0,222,Book03_002,389,4793,808,1075,Even this is eternal morality. They that perfo...,"Write 5 unique questions, corresponding answer...",Question 1: What is the significance of perfor...,"[0.05641840770840645, 0.11018980294466019, -0...."
1,223,Book03_003,390,5453,851,1132,"Section III\n""Vaisampayana said, 'Yudhishthira...","Write 6 unique questions, corresponding answer...",Question 1: What predicament does Yudhishthira...,"[-0.0034319579135626554, 0.08028457313776016, ..."
2,224,Book03_004,392,4383,775,1031,"Section IV\n""Vaisampayana said,--""After the Pa...","Write 5 unique questions, corresponding answer...",Question 1: What was the primary concern expre...,"[-0.015587225556373596, 0.07836046814918518, -..."


# Creat Embedding for Question & Answer


In [30]:
# df_hbqa_auto_qa = pd.read_csv(datapath+"06.1-HBQA_Auto_with_Chunk.csv")
df_hbqa_manual_qa = pd.read_csv(datapath+"06-HBQA_Manual_with_Chunk.csv")


In [59]:
print(df_hbqa_manual_qa.shape)
df_hbqa_manual_qa.head(2)

(1102, 11)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [18]:
DEVICE='cuda'

In [87]:
1696226044/(1000000000)

1.696226044

In [74]:
def get_time(func): 
    import time
    def wrapper(df_hbqa_manual_qa, embmodel):
        start_time = time.time()

        result = func(df_hbqa_manual_qa, embmodel)
        
        end_time = time.time()
        time_spent=end_time-start_time 

        print(f"Time Taken to Process {time_spent}")
        return result
    
    return wrapper

In [82]:
@get_time
def create_sent_vector(df_hbqa_manual_qa, embmodel):

    Ques_Sentences = df_hbqa_manual_qa.Question.tolist()
    Ques_Embeddings = embmodel.encode(Ques_Sentences)

    Ans_Sentences = df_hbqa_manual_qa.Ref_Answer.tolist()
    Ans_Embeddings = embmodel.encode(Ans_Sentences)

    print ("Quest Embedding: ", Ques_Embeddings.shape,"Answer Embedding: ", Ans_Embeddings.shape)
    # return (Ques_Embeddings, Ans_Embeddings)

    return (Ques_Embeddings, Ans_Embeddings) 

In [83]:
Ques_Embeddings, Ans_Embeddings = create_sent_vector(df_hbqa_manual_qa, model)

Quest Embedding:  (1102, 384) Answer Embedding:  (1102, 384)
Time Taken to Process 64.9604754447937


In [20]:
import torch

In [99]:
QuesVectors= torch.tensor(Ques_Embeddings, dtype=torch.float)#.to(DEVICE)
AnsVectors= torch.tensor(Ans_Embeddings, dtype=torch.float)#.to(DEVICE)

In [101]:
df_hbqa_Vector = df_hbqa_manual_qa.copy()

In [23]:
# df_hbqa_Vector.drop(columns = ['WordsInQues','WordsInAns'], axis=1, inplace=True)

In [102]:
# Flatten the tensors into 1D arrays and save
df_hbqa_Vector['QuesVector'] = QuesVectors.tolist()
df_hbqa_Vector['AnsVector'] = AnsVectors.tolist()

In [103]:
df_hbqa_Vector.head(2)

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk,QuesVector,AnsVector
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809,"[-0.05227454751729965, 0.10869505256414413, 0....","[-0.009906493127346039, 0.11283662170171738, -..."
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852,"[0.05215536057949066, 0.11097773164510727, 0.0...","[0.06067391857504845, 0.08465536683797836, -0...."


In [104]:
df_hbqa_Vector.to_csv(datapath + '07.2-HBQA_QA_Vector.csv')

# Save Context/Chunk Vector into Pinecone Vector database

In [105]:
!pip install -Uq pinecone-client

In [ ]:
# import pinecone

# pinecone.init(
# 	api_key='ddf2ef12-9108-4e80-b438-8f86a0b617c0',
# 	environment='asia-southeast1-gcp-free'
# )
# index = pinecone.Index('HBQA')

In [109]:
import os
import pinecone

# # get api key from app.pinecone.io
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# # find your environment next to the api key in pinecone console
# PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key=PINECONE_KEY,
    environment=PINECONE_ENV
)

# index_name = 'semantic-search-fast'
index_name = "hbqa" #it must be in lower letters

In [ ]:
# !pip uninstall pinecone_datasets

In [ ]:
# from pinecone_datasets import load_dataset

# dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# # we drop sparse_values as they are not needed for this example
# dataset.documents.drop(['metadata'], axis=1, inplace=True)
# dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# # we will use 80K rows of the dataset between rows 240K -> 320K
# dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
# dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
# dataset.head()

In [110]:
import ast
import time

In [ ]:
dimension=len(ast.literal_eval(df_hbqa_chunk['ChunkVector'][0])),

In [ ]:
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():

    pinecone.create_index(
        name=index_name,
        dimension=len(df_hbqa_chunk['ChunkVector'][0]),
        metric='cosine'
    )
    # wait a moment for the index to be fully initialized
    time.sleep(1)

In [114]:
# now connect to the index
index = pinecone.Index(index_name)

In [115]:
chunk_id = df_hbqa_chunk['Chunk_Id'].to_list()

chunk_vector = [df_hbqa_chunk['ChunkVector'][i] for i in range(df_hbqa_chunk.shape[0])]

In [117]:
len(chunk_id)

537

In [116]:
import random
import itertools

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

# vector_dim = 128
# vector_count = 10000

# Example generator that generates many (id, vector) pairs
# example_data_generator = map(lambda i: (f'id-{i}', [random.random() for _ in range(vector_dim)]), range(vector_count))
data_generator = map(lambda i: ( 'id'+str(chunk_id[i]), chunk_vector[i]),  range(df_hbqa_chunk.shape[0]))

# Upsert data with 100 vectors per upsert request
i=1
batchsize = 100
for ids_vectors_chunk in chunks(data_generator, batch_size=batchsize):
    index.upsert(vectors=ids_vectors_chunk)  # Assuming `index` defined elsewhere
    print("Processed ",i*batchsize, " Vectors")
    i+=1


Processed  100  Vectors
Processed  200  Vectors
Processed  300  Vectors
Processed  400  Vectors
Processed  500  Vectors
Processed  600  Vectors


# Check whether a correct document can be retrived using a Question

In [124]:
from random import randint
i = randint(0, df_hbqa_chunk.shape[0])
df_hbqa_chunk.loc[i,:]

index                                                                306
Section_Id                                                    Book03_108
Chunk_Id                                                             520
Chunk_Letters                                                       4769
Chunk_Words                                                          837
Chunk_Approx_Tokens                                                 1113
Chunk                  Section CVIII\n"Lomasa said, 'That same king, ...
Prompt                 Write 6 unique questions, corresponding answer...
ChatGPT_Response       Question 1: Why did King Bhagiratha go to the ...
ChunkVector            [-0.07042250782251358, 0.1345033198595047, -0....
Name: 84, dtype: object

In [125]:
# df_hbqa_chunk.loc[i,'ChatGPT_Response']

In [126]:
sep=":"
query = df_hbqa_chunk.loc[i,'ChatGPT_Response'].split('?')[0].split(sep)[1]
print(query)

# create the query vector
xq = model.encode(query).tolist()
topk = 5
# now query
xc = index.query(xq, top_k=topk, include_metadata=True)
xc

 Why did King Bhagiratha go to the snowy Mountain (the Himalayas) and what did he seek there


{'matches': [{'id': 'id520', 'score': 0.580122828, 'values': []},
             {'id': 'id399', 'score': 0.528384447, 'values': []},
             {'id': 'id583', 'score': 0.520113409, 'values': []},
             {'id': 'id432', 'score': 0.508092403, 'values': []},
             {'id': 'id714', 'score': 0.506267667, 'values': []}],
 'namespace': ''}

In [127]:
predictedId=[]
for predid in xc['matches']:
  predictedId.append(predid['id'])

print ("predicted chunkid ", predictedId, " Actual Chunk Id:", df_hbqa_chunk.loc[i,'Chunk_Id'])

predicted chunkid  ['id520', 'id399', 'id583', 'id432', 'id714']  Actual Chunk Id: 520
